<a href="https://colab.research.google.com/github/rrunix/AntecedentesAI/blob/master/MultiCarRacing_RL_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entrenamiento de Agentes con Aprendizaje por Refuerzo en CarRacing-v2**

---

## **Descripcion**

En este notebook aprenderemos a entrenar agentes de **Aprendizaje por Refuerzo (Reinforcement Learning)** utilizando el entorno **CarRacing-v2** de Gymnasium. Este entorno simula una carrera de coches donde un agente debe aprender a conducir por un circuito generado proceduralmente.

### **Que es CarRacing-v2?**

CarRacing-v2 es un entorno clasico de Gymnasium para aprendizaje por refuerzo. Caracteristicas principales:

- **Observaciones**: Imagenes RGB de 96x96 pixeles
- **Acciones**: Espacio continuo con 3 valores [direccion, aceleracion, freno]
- **Recompensas**: +1000/N por cada baldosa visitada, -0.1 por cada frame
- **Objetivo**: Completar el circuito en el menor tiempo posible

### **Libreria utilizada: Stable-Baselines3**

Utilizaremos **Stable-Baselines3**, una libreria de RL que proporciona implementaciones fiables y faciles de usar de algoritmos populares como PPO, A2C, SAC, etc.

### **Algoritmo: PPO (Proximal Policy Optimization)**

PPO es uno de los algoritmos mas populares y efectivos para tareas de control continuo. Sus ventajas incluyen:
- Estabilidad en el entrenamiento
- Buen rendimiento con observaciones de imagenes
- Facil de configurar y ajustar

---

## **1. Instalacion de Dependencias**

Ejecuta las siguientes celdas para instalar todas las librerias necesarias. Esto incluye:
- **Stable-Baselines3**: Libreria de algoritmos de RL
- **Gymnasium**: Entornos de simulacion (incluye CarRacing-v2)
- **Dependencias de visualizacion**: Para renderizar el entorno en Colab

In [ ]:
# Instalacion de dependencias del sistema para renderizado en Colab
# swig es necesario para compilar box2d-py
!apt-get update -qq
!apt-get install -qq -y xvfb python-opengl ffmpeg swig build-essential > /dev/null 2>&1
print("Dependencias del sistema instaladas")

In [ ]:
# Instalacion de librerias de Python
!pip install -q swig
!pip install -q gymnasium[box2d]
!pip install -q stable-baselines3[extra]
!pip install -q pyvirtualdisplay
!pip install -q imageio imageio-ffmpeg
print("Librerias de Python instaladas")

---

## **2. Imports y Configuracion Inicial**

In [ ]:
# Imports principales
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from PIL import Image
import imageio

# Stable-Baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.monitor import Monitor

# Configuracion para visualizacion en Colab (solo si estamos en Colab)
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()
    print("Ejecutando en Google Colab - Display virtual iniciado")
else:
    print("Ejecutando en entorno local")

print("Todas las librerias cargadas correctamente")

In [ ]:
# Crear el entorno
env = gym.make("CarRacing-v3", render_mode="rgb_array")

print("=" * 50)
print("INFORMACION DEL ENTORNO")
print("=" * 50)
print(f"Espacio de observacion: {env.observation_space}")
print(f"Espacio de acciones: {env.action_space}")
print(f"\nForma de observacion: {env.observation_space.shape}")
print(f"Rango de acciones: [{env.action_space.low}, {env.action_space.high}]")
print("\nAcciones:")
print("  - Accion 0: Direccion (steering) [-1, 1]")
print("  - Accion 1: Aceleracion (gas) [0, 1]")
print("  - Accion 2: Freno (brake) [0, 1]")

In [ ]:
# Visualizar una observacion inicial del entorno
obs, info = env.reset()

plt.figure(figsize=(6, 6))
plt.imshow(obs)
plt.title("Observacion inicial del entorno")
plt.axis('off')
plt.show()

print(f"Forma de la observacion: {obs.shape}")
print(f"Tipo de datos: {obs.dtype}")
print(f"Rango de valores: [{obs.min()}, {obs.max()}]")

env.close()

In [ ]:
# Ejecutar algunas acciones aleatorias para ver como funciona
env = gym.make("CarRacing-v3", render_mode="rgb_array")
obs, info = env.reset()

frames = []
total_reward = 0

for step in range(100):
    # Accion aleatoria
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    
    if step % 20 == 0:
        frames.append(obs)
    
    if terminated or truncated:
        break

print(f"Recompensa total con acciones aleatorias: {total_reward:.2f}")

# Mostrar algunos frames
fig, axes = plt.subplots(1, len(frames), figsize=(15, 3))
for i, (ax, frame) in enumerate(zip(axes, frames)):
    ax.imshow(frame)
    ax.set_title(f"Frame {i*20}")
    ax.axis('off')
plt.suptitle("Secuencia de frames con acciones aleatorias")
plt.tight_layout()
plt.show()

env.close()

---

## **4. Preparacion del Entorno para Entrenamiento**

Para entrenar con Stable-Baselines3, necesitamos preparar el entorno correctamente. Usaremos:
- **VecFrameStack**: Apila varios frames consecutivos para dar contexto temporal al agente
- **VecTransposeImage**: Transpone las imagenes al formato esperado por PyTorch (canales primero)

In [ ]:
def make_env():
    """Funcion para crear el entorno con configuracion correcta."""
    env = gym.make("CarRacing-v3", render_mode="rgb_array")
    env = Monitor(env)  # Para registrar estadisticas
    return env

# Crear entorno vectorizado
env = DummyVecEnv([make_env])

# Apilar 4 frames para dar contexto temporal
env = VecFrameStack(env, n_stack=4)

# Transponer imagenes para PyTorch (de HWC a CHW)
env = VecTransposeImage(env)

print(f"Entorno preparado para entrenamiento")
print(f"Forma de observacion: {env.observation_space.shape}")
print(f"(4 frames apilados x 3 canales RGB = 12 canales)")

---

## **5. Configuracion del Modelo PPO**

Ahora configuraremos el modelo PPO con una politica basada en redes neuronales convolucionales (CNN) para procesar las imagenes.

### **Hiperparametros importantes:**
- **learning_rate**: Tasa de aprendizaje (que tan rapido aprende el modelo)
- **n_steps**: Pasos antes de actualizar la politica
- **batch_size**: Tamano del lote para el entrenamiento
- **n_epochs**: Epocas de optimizacion por actualizacion
- **gamma**: Factor de descuento para recompensas futuras

In [ ]:
# Configuracion del modelo PPO
model = PPO(
    policy="CnnPolicy",           # Politica con CNN para imagenes
    env=env,
    learning_rate=3e-4,            # Tasa de aprendizaje
    n_steps=512,                   # Pasos por actualizacion
    batch_size=64,                 # Tamano del batch
    n_epochs=10,                   # Epocas por actualizacion
    gamma=0.99,                    # Factor de descuento
    gae_lambda=0.95,               # GAE lambda para estimacion de ventaja
    clip_range=0.2,                # Rango de clipping de PPO
    verbose=1,                     # Mostrar progreso
    tensorboard_log="./tensorboard_logs/"
)

print("\n" + "=" * 50)
print("MODELO PPO CONFIGURADO")
print("=" * 50)
print(f"Politica: CnnPolicy")
print(f"Learning rate: 3e-4")
print(f"Batch size: 64")
print(f"Gamma (descuento): 0.99")
print(f"Clip range: 0.2")

In [ ]:
# Configurar callbacks para guardar checkpoints durante el entrenamiento
checkpoint_callback = CheckpointCallback(
    save_freq=10000,                    # Guardar cada 10000 pasos
    save_path="./checkpoints/",
    name_prefix="ppo_carracing"
)

print("Callbacks configurados")
print("Los modelos se guardaran en ./checkpoints/")

---

## **6. Entrenamiento**

**Nota importante**: El entrenamiento completo puede tomar varias horas. Para este ejemplo, usaremos un numero reducido de pasos. Para obtener buenos resultados, se recomienda entrenar por al menos 1-2 millones de pasos.

| Timesteps | Tiempo aproximado | Calidad esperada |
|-----------|-------------------|------------------|
| 50,000 | 5-10 min | Demo basica |
| 500,000 | 1-2 horas | Resultados moderados |
| 2,000,000 | 6-8 horas | Buenos resultados |

In [ ]:
# ENTRENAMIENTO
# Ajusta total_timesteps segun el tiempo disponible

TOTAL_TIMESTEPS = 50000  # Cambiar segun necesidad

print(f"Iniciando entrenamiento por {TOTAL_TIMESTEPS:,} pasos...")
print("Esto puede tomar varios minutos.\n")

model.learn(
    total_timesteps=TOTAL_TIMESTEPS,
    callback=checkpoint_callback,
    progress_bar=True
)

print("\nEntrenamiento completado")

In [ ]:
# Guardar el modelo entrenado
model.save("ppo_car_racing_final")
print("Modelo guardado como 'ppo_car_racing_final.zip'")

---

## **7. Evaluacion del Modelo Entrenado**

Vamos a evaluar el rendimiento del agente entrenado y visualizar su comportamiento.

In [ ]:
# Para cargar un modelo guardado previamente:
# model = PPO.load("ppo_car_racing_final")

def evaluate_agent(model, n_episodes=5):
    """Evalua el agente y retorna estadisticas."""
    # Crear entorno de evaluacion
    eval_env = DummyVecEnv([make_env])
    eval_env = VecFrameStack(eval_env, n_stack=4)
    eval_env = VecTransposeImage(eval_env)
    
    episode_rewards = []
    episode_lengths = []
    
    for episode in range(n_episodes):
        obs = eval_env.reset()
        done = False
        total_reward = 0
        steps = 0
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, info = eval_env.step(action)
            total_reward += reward[0]
            steps += 1
            
            if steps > 1000:  # Limite de pasos por episodio
                break
        
        episode_rewards.append(total_reward)
        episode_lengths.append(steps)
        print(f"Episodio {episode + 1}: Recompensa = {total_reward:.2f}, Pasos = {steps}")
    
    eval_env.close()
    
    print("\n" + "=" * 50)
    print("RESULTADOS DE EVALUACION")
    print("=" * 50)
    print(f"Recompensa promedio: {np.mean(episode_rewards):.2f} +/- {np.std(episode_rewards):.2f}")
    print(f"Longitud promedio: {np.mean(episode_lengths):.0f} pasos")
    
    return episode_rewards, episode_lengths

# Evaluar el agente
rewards, lengths = evaluate_agent(model, n_episodes=5)

In [ ]:
# Visualizar resultados de evaluacion
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].bar(range(1, len(rewards) + 1), rewards, color='steelblue')
axes[0].axhline(y=np.mean(rewards), color='red', linestyle='--', label='Promedio')
axes[0].set_xlabel('Episodio')
axes[0].set_ylabel('Recompensa')
axes[0].set_title('Recompensa por Episodio')
axes[0].legend()

axes[1].bar(range(1, len(lengths) + 1), lengths, color='forestgreen')
axes[1].axhline(y=np.mean(lengths), color='red', linestyle='--', label='Promedio')
axes[1].set_xlabel('Episodio')
axes[1].set_ylabel('Pasos')
axes[1].set_title('Longitud del Episodio')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Grabar un video del agente
def record_video(model, video_path="agent_video.mp4", max_steps=500):
    """Graba un video del agente jugando."""
    # Crear entorno para grabacion
    video_env = DummyVecEnv([make_env])
    video_env = VecFrameStack(video_env, n_stack=4)
    video_env = VecTransposeImage(video_env)
    
    # Entorno sin procesar para obtener frames visuales
    raw_env = gym.make("CarRacing-v2", render_mode="rgb_array")
    
    frames = []
    obs = video_env.reset()
    raw_obs, _ = raw_env.reset()
    
    for step in range(max_steps):
        # Renderizar frame
        frame = raw_env.render()
        frames.append(frame)
        
        # Obtener accion del modelo
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = video_env.step(action)
        raw_obs, _, terminated, truncated, _ = raw_env.step(action[0])
        
        if done[0] or terminated or truncated:
            break
    
    video_env.close()
    raw_env.close()
    
    # Guardar video
    imageio.mimsave(video_path, frames, fps=30)
    print(f"Video guardado en: {video_path}")
    return video_path

# Grabar video
video_path = record_video(model, max_steps=300)

In [ ]:
# Mostrar video en el notebook
from IPython.display import Video
Video(video_path, embed=True, width=400)

---

## **8. Uso con Otros Entornos**

Una de las grandes ventajas de **Stable-Baselines3** es que el mismo codigo puede utilizarse con muchos otros entornos de Gymnasium. A continuacion se muestran ejemplos de como adaptar este notebook para otros entornos populares.

### **Entornos Clasicos de Control**

Estos entornos son mas simples y entrenan mas rapido, ideales para experimentar:

In [ ]:
# Ejemplo 1: CartPole-v1 (Control discreto)
# Un pendulo que debe mantenerse en equilibrio

cartpole_env = gym.make("CartPole-v1")
print("CartPole-v1:")
print(f"  Observacion: {cartpole_env.observation_space}")
print(f"  Acciones: {cartpole_env.action_space} (0=izquierda, 1=derecha)")
print("\nComo entrenar:")
print('  model = PPO("MlpPolicy", cartpole_env, verbose=1)')
print('  model.learn(total_timesteps=50000)')

cartpole_env.close()

In [ ]:
# Ejemplo 2: LunarLander-v2 (Control discreto mas complejo)
# Aterrizar una nave espacial en la luna

lunar_env = gym.make("LunarLander-v2")
print("LunarLander-v2:")
print(f"  Observacion: {lunar_env.observation_space}")
print(f"  Acciones: {lunar_env.action_space}")
print("  (0=nada, 1=motor izq, 2=motor principal, 3=motor der)")
print("\nComo entrenar:")
print('  model = PPO("MlpPolicy", lunar_env, verbose=1)')
print('  model.learn(total_timesteps=500000)')

lunar_env.close()

In [ ]:
# Ejemplo 3: BipedalWalker-v3 (Control continuo)
# Un robot bipedo que debe aprender a caminar

walker_env = gym.make("BipedalWalker-v3")
print("BipedalWalker-v3:")
print(f"  Observacion: {walker_env.observation_space}")
print(f"  Acciones: {walker_env.action_space}")
print("  (4 valores continuos para controlar las articulaciones)")
print("\nComo entrenar (requiere mas tiempo):")
print('  model = PPO("MlpPolicy", walker_env, verbose=1)')
print('  model.learn(total_timesteps=2000000)')

walker_env.close()

### **Entornos Atari (Observaciones de Imagenes)**

Para juegos Atari, se utiliza `CnnPolicy` similar a CarRacing:

In [ ]:
# Ejemplo 4: Juegos Atari
# Nota: Requiere instalar ale-py y roms adicionales

print("Para entrenar en juegos Atari:")
print("")
print("1. Instalar dependencias:")
print("   !pip install ale-py")
print("   !pip install autorom && AutoROM --accept-license")
print("")
print("2. Codigo de ejemplo para Breakout:")
print("   from stable_baselines3.common.atari_wrappers import AtariWrapper")
print("   ")
print('   env = gym.make("ALE/Breakout-v5")')
print('   env = AtariWrapper(env)')
print('   env = DummyVecEnv([lambda: env])')
print('   env = VecFrameStack(env, n_stack=4)')
print('   ')
print('   model = PPO("CnnPolicy", env, verbose=1)')
print('   model.learn(total_timesteps=1000000)')

### **Tabla de Referencia: Politicas por Tipo de Entorno**

| Tipo de Entorno | Observacion | Politica | Ejemplo |
|-----------------|-------------|----------|----------|
| Control clasico | Vector numerico | `MlpPolicy` | CartPole, LunarLander |
| Imagenes | RGB/Grayscale | `CnnPolicy` | Atari, CarRacing |
| Multiples inputs | Dict/Tuple | `MultiInputPolicy` | Entornos personalizados |

### **Otros Algoritmos Disponibles**

Ademas de PPO, Stable-Baselines3 ofrece otros algoritmos que pueden ser mas adecuados segun el problema:

In [ ]:
print("Algoritmos disponibles en Stable-Baselines3:")
print("")
print("PPO (Proximal Policy Optimization)")
print("  - Bueno para todo tipo de problemas")
print("  - Estable y facil de configurar")
print('  - Uso: PPO("MlpPolicy", env)')
print("")
print("A2C (Advantage Actor-Critic)")
print("  - Rapido, bueno para problemas simples")
print("  - Menos estable que PPO")
print('  - Uso: A2C("MlpPolicy", env)')
print("")
print("SAC (Soft Actor-Critic)")
print("  - Excelente para control continuo")
print("  - Muy eficiente en muestras")
print('  - Uso: SAC("MlpPolicy", env)')
print("")
print("DQN (Deep Q-Network)")
print("  - Solo para acciones discretas")
print("  - Clasico de RL profundo")
print('  - Uso: DQN("MlpPolicy", env)')
print("")
print("TD3 (Twin Delayed DDPG)")
print("  - Control continuo deterministico")
print("  - Similar a SAC, puede ser mas estable")
print('  - Uso: TD3("MlpPolicy", env)')

---

## **9. Conclusiones**

En este notebook hemos aprendido:

1. **Configurar el entorno CarRacing-v2** de Gymnasium para entrenamiento de RL
2. **Utilizar Stable-Baselines3** para implementar el algoritmo PPO de forma sencilla
3. **Entrenar y evaluar** un agente de aprendizaje por refuerzo
4. **Adaptar el codigo** para otros entornos de Gymnasium

### **Proximos pasos sugeridos:**

- Entrenar por mas tiempo (1-2 millones de pasos) para mejores resultados
- Experimentar con diferentes hiperparametros
- Probar otros algoritmos como SAC o A2C
- Usar TensorBoard para visualizar el progreso del entrenamiento:
  ```
  %load_ext tensorboard
  %tensorboard --logdir ./tensorboard_logs/
  ```

### **Recursos adicionales:**

- [Documentacion de Stable-Baselines3](https://stable-baselines3.readthedocs.io/)
- [Gymnasium](https://gymnasium.farama.org/)
- [Tutorial de RL de OpenAI](https://spinningup.openai.com/)

In [ ]:
# Limpieza final
env.close()
print("Notebook completado exitosamente")